### 2014-01 2do Recuperatorio
Se tiene un archivo distribuido con información de una red social en la cual cada registro tiene información sobre un usuario y una lista de sus amigos y sus enemigos (user_id,vector_ids_amigos,vector_ids_enemigos). Queremos encontrar al usuario que figura en mas listas de enemigos, para designarlo el enemigo público número uno.
(8 pts por programar lo pedido) (5 pts por programar la agregación necesaria) (2 pts indique cuantos procesos reducers usaría)

In [3]:
# (user_id,vector_ids_amigos,vector_ids_enemigos)
social_net = [
    (1, [2, 3], [4, 5, 6]),
    (2, [3, 4], [5, 6]),
    (3, [2, 1], [7, 6]),
    (4, [6, 7, 1], [6]),
    (5, [1, 2, 3, 4], [7]),
    (6, [7], [1, 2, 3, 4]),
    (7, [6], [1, 5, 4, 6])
]

In [14]:
# Usuario que figura en mas listas de enemigos
socialRDD = sc.parallelize(social_net)
socialRDD.flatMap(lambda x: x[2])\
         .map(lambda x: (x, 1))\
         .reduceByKey(lambda a, b: a + b)\
         .reduce(lambda a, b: a if a[1] > b[1] else b)[0]

6

---
### 2016-1 2do Recuperatorio
Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id,user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón. 

In [1]:
# (chat_id,user_id, nickname, text)
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [5]:
# Usuario mas pregunton
chatRDD = sc.parallelize(chats)
chatRDD.map(lambda x: (x[1], x[3].count('?')))\
       .reduceByKey(lambda a, b: a + b)\
       .reduce(lambda a, b: a if a[1] > b[1] else b)[0]

1